In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
import classifier as cf
import seaborn as sns
sns.set_style("darkgrid")

In [2]:
traindata = pd.read_csv("data/processed_training_data.csv", nrows=200000)
traindata.shape

(200000, 26)

In [3]:
def give_score(row):
    if row["booking_bool"] > 0:
        return int(5)
    if row["click_bool"] > 0:
        return int(1)
    return int(0)

In [4]:
traindata["score"] = traindata.apply(give_score, axis=1)
max(traindata["score"])

5

In [5]:
testdata = pd.read_csv("data/processed_testing_data.csv", nrows=50000)
testdata.shape

(50000, 23)

In [6]:
print(traindata.columns)

Index(['Unnamed: 0', 'srch_id', 'date_time', 'site_id',
       'visitor_location_country_id', 'prop_country_id', 'prop_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'random_bool', 'click_bool', 'booking_bool', 'score'],
      dtype='object')


In [7]:
attrs = ["prop_starrating", "prop_location_score1", "price_usd"]
y = traindata["score"]
X = traindata[attrs]
flag = "True"
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=101)

In [8]:
Xtest = testdata[attrs]

In [9]:
gnb = cf.gaussian_naive_bayes(Xtrain, Xtest, ytrain, ytest, flag)

Gaussian Naive Bayes: cross vali score 0.9549533333333334 , accuracy score 0.95606


In [10]:
dt = cf.decision_tree(Xtrain, Xtest, ytrain, ytest, flag)

Decision Tree: cross vali score 0.9289733333333333 , accuracy score 0.93086


In [11]:
knn = cf.k_nearest_neighbours(Xtrain, Xtest, ytrain, ytest, flag)

K Nearest Neighbours: cross vali score 0.9546466666666668 , accuracy score 0.95592


In [12]:
sgd = cf.stochastic_gradient_descent(Xtrain, Xtest, ytrain, ytest, flag)

Stochastic Gradient Descent: cross vali score 0.9488666666666667 , accuracy score 0.89362


In [13]:
lr = cf.logistic_regression(Xtrain, Xtest, ytrain, ytest, flag)

C:\Users\Helia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Helia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

KeyboardInterrupt: 

In [16]:
testdata["score"] = gnb.tolist()
testdata = testdata.sort_values(["srch_id", "score"])
df = pd.DataFrame({"srch_id": testdata["srch_id"].values,
                   "prop_id": testdata["prop_id"]})
df.to_csv("submission.csv", index=False)
print("Yes")

Yes
